In [118]:
import json
from sys import argv
import re


def checker(nm,ori):
    flag = False
    for key in ori.keys():
        if ori[key] != nm[key]:
            flag = True
            break
    print('Match!!' if flag == False else 'Error!!')

def test(origin_mapping,idx):
    print(len(idx))
    mark = consecu_int_run_length(idx)
    # check part
    _m = mark
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    checker(nm,origin_mapping)

    sporadic_seg = sporadic_seg_run_length(mark)
    # check part
    _m = sporadic_seg
    nm = {}
    vv = set(values)
    for i in range(len(keys)):
        if keys[i] in values:
            nm[keys[i]] = keys[i]
        #nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        if _m[i+2] != 1:
            for j in range(_m[i+2]):
                nm[keys[_m[i+1]+j]] = values[_m[i]+j]
        else:
            length = len(_m[i+1])
            for j in range(length):
                nm[keys[_m[i+1][j]]] = values[_m[i]+j]
    checker(nm,origin_mapping)

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = []
    for i in range(len(k)):
        if i != idx[i]:      
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                temp = []
                flag = False
    if len(temp) != 0:
        mark += temp
    return mark

def sporadic_seg_run_length(mark):
    sporadic_seg = [ mark[i:i+3] for i in range(0,len(mark),3)]
    flag = False
    temp = []
    res = []
    for i in range(len(sporadic_seg)):
        if sporadic_seg[i][2] != 1:
            if flag == True:
                res += temp 
                # initialize temp to an empty list, to avoid if temp have something but index is at the end
                temp = []
                flag = False
            res += sporadic_seg[i]
        else:
            # first one
            if flag == False:
                # start index, index list, len
                temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
                flag = True
            else:
                if sporadic_seg[i][0] - sporadic_seg[i-1][0] == 1:
                    temp[1].append(sporadic_seg[i][1]) 
                else:
                    res += temp  
                    temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
    if len(temp) != 0:
        res += temp  
    return res

def classify(origin_mapping):
    same, prefix, infix, postfix, others = [], [], [], [], []
    for k,v in origin_mapping.items():
        index = v.find(k)
        if k == v:
            same += [k]
        elif index != -1:
            if index == 0:
                prefix += [k]
            elif v[index:] == k:
                postfix += [k]
            else:
                infix += [k]
        else:
            others += [k]
    print('same: {}\nprefix: {}\ninfix: {}\npostfix: {}\nothers: {}'.format(len(same),len(prefix),len(infix),len(postfix),len(others)))
    return same, prefix, infix, postfix, others

def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K:
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

def mycmp(rhs,lhs):
    col=0
    m1 = re.findall('\[([0-9]+)\]',rhs[col])
    m2 = re.findall('\[([0-9]+)\]',lhs[col])
    if len(m1) == 0 or len(m2) == 0:
        l = min(len(rhs[col]),len(lhs[col]))
        for i in range(l):
            if rhs[col][i] > lhs[col][i]:
                return 1
            elif rhs[col][i] == lhs[col][i]:
                continue
            else:
                return -1
        return 1 if l == len(rhs[col]) else 0
    else:  
        m1 = list(re.finditer('\[([0-9]+)\]',rhs[col]))
        m2 = list(re.finditer('\[([0-9]+)\]',lhs[col]))
        st1 = m1[0].span()
        st2 = m2[0].span()
    
        if rhs[col][:st1[0]] > lhs[col][:st2[0]]:
            return 1
        elif rhs[col][:st1[0]] < lhs[col][:st2[0]]:
            return -1
        else:
            l = min(len(m1),len(m2))
            flag=0
            for i in range(l):
                flag=int(m1[i].group(1))-int(m2[i].group(1))
                if flag == 0:
                    continue
                else: 
                    return flag
            if rhs[col][st1[1]:]>lhs[col][st2[1]:]:
                return 1
            elif rhs[col][st1[1]:]==lhs[col][st2[1]:]:
                return 0
            else:
                return -1

if __name__ == '__main__':
    case = 3
    
    
    map_in_name, script_name = '../cases/case{}.json'.format(case), 'test.py'
    nf = '../input/case{}_in.json'.format(case)
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()
    # rule based classifying
    same, prefix, infix, postfix, others = classify(origin_mapping)
    # only remove same
    same_set = set(same)
    exclusive_same = dict([item for item in origin_mapping.items() if item[0] not in same_set])
    k,v=zip(*origin_mapping.items())
    test = list(zip(k,v))
    test = sorted(test,key=cmp_to_key(mycmp))
    #for i in range(100):
        #print(test[i])
    k,v = zip(*test)
    k = list(k)
    v = list(v) 
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=cmp_to_key(mycmp))
   
    print('total: {}'.format(len(k)))
    ccc = 0
    for i in range(len(k)):
        if i != mm[i][1]:
            ccc+=1
    print('count of wrong idx: {}'.format(ccc))
    v, idx = zip(*mm) 
    idx = list(idx)
    v = list(v)
    mark = consecu_int_run_length(idx)
    print(len(mark))
    max_consecutive_int = 0
    for i in range(0,len(mark),3):
        if mark[i+2] < 2:
            max_consecutive_int+=1
    print('max consecutive ints: {}'.format(max_consecutive_int))
    sporadic_seg = sporadic_seg_run_length(mark)
    print('length of sporadic_seg: {}'.format(len(sporadic_seg)))
    print('str len of sporadic_seg: {}'.format(len(str(sporadic_seg))))
    #mmm = sorted(list(exclusive_same.items()),key=lambda x: x[0])
    #mmm = sorted(list(exclusive_same.items()),key=cmp_to_key(mycmp))
    document1 = """import json as a,sys 
w={}
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
k,v=set(k),set(v)
r=k&v
c=list(r)
m=dict(zip(c,c))
k,v=s(k-r),s(v-r)
l=len
g=range
for i in g(l(k)):
    m[k[i]]=v[i]
for i in g(0,l(w),3):
    if w[i+2]!=1:
        for j in g(w[i+2]):
             m[k[w[i+1]+j]]=v[w[i]+j]
    else:
        for j in g(l(w[i+1])):
            m[k[w[i+1][j]]]=v[w[i]+j]
a.dump(m,open(o,'w'))""".format(str(sporadic_seg))

    document2 = """import json as a,sys
n,o=sys.argv[1:3]
k,v=a.load(open(n))
s=sorted
a.dump(dict(zip(s(k),s(v))),open(o,'w'))"""

    #script=open(script_name,'w')
    #script.write(document1 if len(sporadic_seg) != 0 else document2)
    #script.close()

same: 3073
prefix: 7212
infix: 0
postfix: 0
others: 52441
total: 62726
count of wrong idx: 40736
20985
max consecutive ints: 2365
length of sporadic_seg: 20853


In [52]:
def mycmp(rhs,lhs):
    col=0
    m1 = re.findall('\[([0-9]+)\]',rhs[col])
    m2 = re.findall('\[([0-9]+)\]',lhs[col])
    if len(m1) == 0 or len(m2) == 0:
        l = min(len(rhs[col]),len(lhs[col]))
        for i in range(l):
            if rhs[col][i] > lhs[col][i]:
                return 1
            elif rhs[col][i] == lhs[col][i]:
                return 0
            else:
                return -1
    else:  
        m1 = list(re.finditer('\[([0-9]+)\]',rhs[col]))
        m2 = list(re.finditer('\[([0-9]+)\]',lhs[col]))
        st1 = m1[0].span()[0]
        st2 = m2[0].span()[0]
        if rhs[col][:st1] > lhs[col][:st2]:
            return 1
        elif rhs[col][:st1] < lhs[col][:st2]:
            return -1
        else:
            flag=0
            for i in range(len(m1)):
                flag=int(m1[i].group(1))-int(m2[i].group(1))
                if flag == 0:
                    continue
                else: 
                    return flag
k,v=zip(*origin_mapping.items())
test = list(zip(k,v))
test = sorted(test,key=cmp_to_key(mycmp))
for i in range(0,1030,1):
    print(test[i])

('CARPE_beek_qv_ole_0_4/s_qid', 'CARPE_beek_qv_ole_0_4/s_qid/frey2')
('CARPE_beek_qv_ole_0_0/s_qid', 'CARPE_beek_qv_ole_0_0/s_qid/frey2')
('CARPE_beek_qv_ole_0_8/s_qid', 'CARPE_beek_qv_ole_0_8/s_qid/frey2')
('CARPE_beek_qv_ole_0_1/s_qid', 'CARPE_beek_qv_ole_0_1/s_qid/frey2')
('CARPE_beek_qv_ole_0_2/s_qid', 'CARPE_beek_qv_ole_0_2/s_qid/frey2')
('CARPE_beek_qv_ole_0_3/s_qid', 'CARPE_beek_qv_ole_0_3/s_qid/frey2')
('CARPE_beek_qv_ole_0_5/s_qid', 'CARPE_beek_qv_ole_0_5/s_qid/frey2')
('CARPE_beek_qv_ole_0_6/s_qid', 'CARPE_beek_qv_ole_0_6/s_qid/frey2')
('CARPE_beek_qv_ole_0_7/s_qid', 'CARPE_beek_qv_ole_0_7/s_qid/frey2')
('DBMS_URUSES_BAA_BEDS', 'DBMS_URUSES_BAA_BEDS')
('Hexarch_UN_PTT_DODDED_SHAKHA', 'Hexarch_UN_PTT_DODDED_SHAKHA')
('Hexarch_UN_PTT_DODDED_SHAKHA_0', 'Hexarch_UN_PTT_DODDED_SHAKHA_0')
('arecuna_ungt[3]', 'arecuna_ungt[3]')
('arecuna_ungt[4]', 'arecuna_ungt[4]')
('arecuna_ungt[5]', 'arecuna_ungt[5]')
('arecuna_ungt[6]', 'arecuna_ungt[6]')
('arecuna_ungt[7]', 'arecuna_ungt[7]')
(